# Gensim Topic Modeling

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

LDA’s approach to topic modeling is it considers each document as a collection of topics in a certain proportion. And each topic as a collection of keywords, again, in a certain proportion.

In [40]:
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.hdpmodel import HdpModel # Heirarichal Dirichlet process
from gensim.models.wrappers import LdaVowpalWabbit, LdaMallet
from gensim.corpora.dictionary import Dictionary
from numpy import array

In [41]:
texts = [['human', 'interface', 'computer'],
         ['survey', 'user', 'computer', 'system', 'response', 'time'],
         ['eps', 'user', 'interface', 'system'],
         ['system', 'human', 'system', 'eps'],
         ['user', 'response', 'time'],
         ['trees'],
         ['graph', 'trees'],
         ['graph', 'minors', 'trees'],
         ['graph', 'minors', 'survey']]

In [43]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(texts, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[texts], threshold=100)  


In [46]:
# The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus.
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts] # # Term Document Frequency

In [47]:
corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

In [48]:
# with 50 iteration
goodLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=50, num_topics=2)

# one iteration ony
badLdaModel = LdaModel(corpus=corpus, id2word=dictionary, iterations=1, num_topics=2)

### Topic Coherence
Topic Coherence is a measure used to evaluate topic models: methods that automatically generate topics from a collection of documents, using latent variable models.
Each such generated topic consists of words, and the topic coherence is applied to the top N words from the topic. It is defined as the average / median of the pairwise word-similarity scores of the words in the topic (e.g. PMI).

A good model will generate coherent topics, i.e., topics with high topic coherence scores. Good topics are topics that can be described by a short label, therefore this is what the topic coherence measure should capture.

I'm not an expert, but I think the scores are mainly comparative: if topic B has a higher coherence score than topic A, it is 'better' (more coherent).

In [49]:
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')
badcm = CoherenceModel(model=badLdaModel, texts=texts, dictionary=dictionary, coherence='c_v')
print(goodcm.get_coherence(),badcm.get_coherence())

0.3838413553737203 0.3838413553737203


In [50]:
goodcm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=dictionary, coherence='u_mass')
badcm = CoherenceModel(model=badLdaModel, texts=texts, dictionary=dictionary, coherence='u_mass')
print(goodcm.get_coherence(),badcm.get_coherence())

-14.664627001047737 -14.726061108336854


In [53]:
# Compute Perplexity
goodLdaModel.log_perplexity(corpus)  # a measure of how good the model is. lower the better.

-3.0212114083512485

## Visualize topic models

Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

In [35]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(goodLdaModel, corpus, dictionary)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.054003  0.0       1        1  63.328152
0     -0.054003  0.0       2        1  36.671848, topic_info=         Term      Freq     Total Category  logprob  loglift
10      graph  2.000000  2.000000  Default  12.0000  12.0000
9       trees  2.000000  2.000000  Default  11.0000  11.0000
11     minors  1.000000  1.000000  Default  10.0000  10.0000
4      survey  2.000000  2.000000  Default   9.0000   9.0000
5      system  3.000000  3.000000  Default   8.0000   8.0000
0    computer  2.000000  2.000000  Default   7.0000   7.0000
7        user  3.000000  3.000000  Default   6.0000   6.0000
2   interface  2.000000  2.000000  Default   5.0000   5.0000
8         eps  2.000000  2.000000  Default   4.0000   4.0000
1       human  2.000000  2.000000  Default   3.0000   3.0000
3    response  2.000000  2.000000  Default   2.0000   2.0000
6        time  2.000000  2.000000  Default   1.0000   1.0000
5      system  3.312271  3.818567   Topic1  -1.7128   0.3146
7        user  2.556311  3.030272   Topic1  -1.9719   0.2868
8         eps  1.861117  2.257952   Topic1  -2.2893   0.2636
3    response  1.746159  2.227730   Topic1  -2.3530   0.2133
6        time  1.736011  2.225062   Topic1  -2.3589   0.2086
1       human  1.423402  2.142880   Topic1  -2.5574   0.0477
2   interface  1.338364  2.120524   Topic1  -2.6190  -0.0034
0    computer  1.191322  2.081868   Topic1  -2.7354  -0.1014
4      survey  1.043989  2.043135   Topic1  -2.8674  -0.2146
11     minors  0.685614  1.948921   Topic1  -3.2879  -0.5879
9       trees  0.805754  2.570065   Topic1  -3.1264  -0.7031
10      graph  0.664848  2.533021   Topic1  -3.3187  -0.8808
10      graph  1.868173  2.533021   Topic2  -1.7392   0.6987
9       trees  1.764311  2.570065   Topic2  -1.7964   0.6270
11     minors  1.263307  1.948921   Topic2  -2.1304   0.5696
4      survey  0.999146  2.043135   Topic2  -2.3650   0.2878
0    computer  0.890546  2.081868   Topic2  -2.4801   0.1540
2   interface  0.782160  2.120524   Topic2  -2.6098   0.0058
1       human  0.719478  2.142880   Topic2  -2.6934  -0.0882
6        time  0.489051  2.225062   Topic2  -3.0794  -0.5119
3    response  0.481571  2.227730   Topic2  -3.0948  -0.5285
8         eps  0.396835  2.257952   Topic2  -3.2884  -0.7355
7        user  0.473961  3.030272   Topic2  -3.1108  -0.8521
5      system  0.506297  3.818567   Topic2  -3.0448  -1.0173, token_table=      Topic      Freq       Term
term                            
0         1  0.480338   computer
0         2  0.480338   computer
8         1  0.885759        eps
10        1  0.394785      graph
10        2  0.789571      graph
1         1  0.466662      human
1         2  0.466662      human
2         1  0.471582  interface
2         2  0.471582  interface
11        1  0.513104     minors
11        2  0.513104     minors
3         1  0.897775   response
4         1  0.489444     survey
4         2  0.489444     survey
5         1  0.785635     system
5         2  0.261878     system
6         1  0.898851       time
9         1  0.389095      trees
9         2  0.778190      trees
7         1  0.990010       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [60]:
# save html file
vis = pyLDAvis.gensim.prepare(goodLdaModel, corpus, dictionary)
vis = pyLDAvis.prepared_data_to_html(vis)

with open("LDA_output.html", "w") as file:
    file.write(vis)

In [37]:
topics = []
for topic_id, topic in goodLdaModel.show_topics(num_topics=10,formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [38]:
topics

[['graph',
  'trees',
  'minors',
  'survey',
  'computer',
  'interface',
  'human',
  'system',
  'time',
  'response'],
 ['system',
  'user',
  'eps',
  'response',
  'time',
  'human',
  'interface',
  'computer',
  'survey',
  'trees']]

# Mallet’s version of LDA algorithm

In [57]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = r'E:\projects\mallet-2.0.8\bin\mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=dictionary)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Sabeeha\\AppData\\Local\\Temp\\9d5505_state.mallet.gz'

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

### How to find the optimal number of topics for LDA?
My approach to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value.